In [1]:
import open3d as o3d
import numpy as np
import math
from scipy.spatial import Delaunay
import matplotlib.pyplot as plt
import math
from functools import reduce
import os

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [296]:
pc_path = os.path.join('..','data','dataset','custom','train','heap_example.ply')
pcd = o3d.io.read_point_cloud(pc_path)
# pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
axes = o3d.geometry.TriangleMesh.create_coordinate_frame()

In [297]:
o3d.visualization.draw_geometries([pcd, axes])

# Generating variants

In [298]:
def gen_single_pcd_variant(pc_path, axis = 1, max_mode = "max", margin = 0.01):
    pcd = o3d.io.read_point_cloud(pc_path)
    pcd_pts =  np.asarray(pcd.points)
    base_min = np.min(pcd_pts[:,axis])
    if max_mode == "min":
        if base_min < 0:
            base_max = -margin*abs(base_min)
        else:
            base_max = (1+margin)*base_min
    else:
        base_max = abs(np.random.randn())*np.max(pcd_pts[:,axis])
    base_idx = np.where((pcd_pts[:,axis] >= base_min) & (pcd_pts[:,axis] <= base_max))[0]
    base_pts = pcd_pts[base_idx,:]

    mean_x, std_x = np.random.randint(low=1,high=16)*np.mean(base_pts[:,0]), max(0.5,abs(0.5*np.random.randn()))*np.std(base_pts[:,0])
    mean_y, std_y = 2.0*np.mean(base_pts[:,1]), 0.01*np.std(base_pts[:,1])
    mean_z, std_z = np.random.randint(low=1,high=16)*np.mean(base_pts[:,2]), max(0.5,abs(0.5*np.random.randn()))*np.std(base_pts[:,2])

    noise_x = mean_x + std_x*np.random.randn(base_pts[:,0].shape[0])
    noise_y = 0 #+ mean_y + std_z*np.random.randn(base_pts[:,0].shape[0])
    noise_z = mean_z + std_z*np.random.randn(base_pts[:,0].shape[0])

    base_pts_noise = np.zeros_like(base_pts)
    base_pts_noise[:,0] =  base_pts[:,0] + noise_x
    base_pts_noise[:,1] =  base_pts[:,1] + noise_y
    base_pts_noise[:,2] =  base_pts[:,2] + noise_z

    pcd_pts[base_idx,:] = base_pts_noise
    pcd_noise = o3d.geometry.PointCloud()
    pcd_noise.points = o3d.utility.Vector3dVector(pcd_pts)
    
    return pcd_noise

def gen_pcd_variants(pc_path, num_variants = 5, axis = 1, max_mode = "max", margin = 0.01):
    pcd_variants = []
    for i in range(num_variants):
        pcd_variant = gen_single_pcd_variant(pc_path, axis = axis, max_mode = max_mode, margin = margin)
        pcd_variants.append(pcd_variant)
    return pcd_variants

In [299]:
pc_path = os.path.join('..','data','dataset','custom','train','heap.ply')
num_variants = 10
max_mode = "max" # "max" or "min"
margin = 0.01 #between 0 and 1
axis = 1
pcd_variants = gen_pcd_variants(pc_path, num_variants, axis, max_mode, margin)

In [300]:
dir_path = f"{os.sep}".join(pc_path.split(os.sep)[:-1])
fname = pc_path.split(os.sep)[-1]
for i in range(num_variants):
    variant_fname = f"{fname.split('.')[0]}_{i}.{fname.split('.')[1]}"
    save_path = os.path.join(dir_path, variant_fname)
    o3d.io.write_point_cloud(save_path, pcd_variants[i])
    # o3d.visualization.draw_geometries([pcd_variants[i]])